To create a notion -> lancedb pipeline, we need to install:
1. dlt with lancedb extras
2. sentence-transformers: we need to use an embedding model to vectorize and store data inside LanceDB. For this we choose the open-source model "sentence-transformers/all-MiniLM-L6-v2".

In [ ]:
%%capture
!pip install requirements.txt

We now create a dlt project using the command `dlt init <source> <destination>`.

This downloads all the modules required for the dlt source (rest api, in this case) into the local directory. See the side panel for the directory structure created.

What is the dlt rest api source?

It is a dlt source that allows you to connect to any REST API endpoint using a declarative configuration. You can:
- pass the endpoints that you want to connect to,
- define the relation between the endpoints
- define how you want to handle pagination and authentication

In [ ]:
!yes | dlt init rest_api lancedb

In [1]:
from dotenv import load_dotenv
import os

_ = load_dotenv()

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

In [2]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

from datetime import datetime, timezone

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="employee_handbook")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "homework",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config,name="employee_handbook")

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="employee_handbook",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    # last_value = last_edited_time.last_value
    # print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block

def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="employee_handbook",
        write_disposition="merge"
    )
    print(load_info)

load_notion()

/Users/fustin/Desktop/datatalks-llm-zoomcamp/workshops/dlt_lancedb/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pipeline company_policies load step completed in 6.07 seconds
1 load package(s) were loaded to destination LanceDB and into dataset notion_pages
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x13ebe3560> location to store data
Load package 1721608322.382782 is LOADED and contains no failed jobs


In [3]:
import lancedb

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___employee_handbook")

data_df = dbtable.to_pandas()
data_df

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,f389f97c-03aa-5271-b2fb-051ce16574e0,"[-0.024265679, 0.047460854, -0.0117965005, 0.0...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-22 00:32:05.038466+00:00,1721608322.382782,py80agBX6MyXxQ
1,a9de527a-f0e9-50e9-9817-2df0ef208ba5,"[-0.049661625, 0.10853516, -0.009762597, -0.03...",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-22 00:32:05.041815+00:00,1721608322.382782,2U4rgwpRFa7tkA
2,72ac35bc-9e93-56cc-a50d-d5ebe3232d7d,"[-0.063163206, 0.17331511, 0.0253517, -0.01914...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-22 00:32:05.041953+00:00,1721608322.382782,P3Y+EAU3qpmi/Q
3,c595a6a7-40fe-5ce4-8a3c-37840f8c2f97,"[-0.10974316, 0.1058608, 0.0032906258, -0.0213...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-22 00:32:05.042034+00:00,1721608322.382782,L+vSYYbjk3wHCg
4,423f75c2-d53b-5d21-8e71-44f66bb4fc52,"[0.052423306, -0.064575955, 0.06586305, 0.0145...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-22 00:32:05.042111+00:00,1721608322.382782,NCBjz+xEysmJCw
5,5b0983ba-a662-504d-959b-019c8ecc19a8,"[0.0005233787, -0.054883447, 0.043573413, -0.0...",e4ec9f4d-b687-4c28-a80d-985bfabcc2ba,paragraph,Remote working refers to working from a non-of...,2024-07-05 22:52:00+00:00,2024-07-22 00:32:05.042192+00:00,1721608322.382782,DDKXMwZt+neh/g
6,9c6129ca-8c44-5eef-b513-1159281b35ef,"[0.038026273, -0.02150966, 0.047527824, 0.0647...",e6e550dc-b59e-4928-abd7-07eace948681,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:52:00+00:00,2024-07-22 00:32:05.042269+00:00,1721608322.382782,JEzAytRrdI+sfg
7,18049710-1a19-5b38-a44f-b5e14761cc04,"[-0.058588054, -0.07540447, 0.033775195, 0.009...",a269d0ca-ce14-481b-a5f4-9192d6840d6e,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-05 22:52:00+00:00,2024-07-22 00:32:05.042347+00:00,1721608322.382782,7LxTk5GFkow1Tg
8,04577168-5479-5fd0-9534-85a151fd0645,"[-0.013599243, 0.0047530434, 0.024835154, 0.01...",5b65f3e7-0a37-429a-818d-f99b53755ebd,paragraph,"In this section, we are going to be covering i...",2024-07-05 23:33:00+00:00,2024-07-22 00:32:05.042426+00:00,1721608322.382782,LqDmyuFMYN9QMQ
9,d7225a37-08ef-528a-b8ee-a2d6c6a86e3b,"[0.032060876, 0.024244612, 0.0084713185, 0.031...",b27f7d80-f2f1-460e-aa0c-b8e770cf050a,paragraph,Our company observes the following holidays: N...,2024-07-05 22:52:00+00:00,2024-07-22 00:32:05.042495+00:00,1721608322.382782,8MKi0sBj7UHREg


In [4]:
len(data_df)

17

In [5]:
data_df.last_edited_time.mean()

Timestamp('2024-07-05 22:52:52.941176+0000', tz='UTC')

In [6]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

```bash
docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
docker exec -it ollama bash
ollama pull phi3
```

In [7]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [9]:
question = "how many PTO days are the employees entitled to in a year?"

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___employee_handbook")
context = retrieve_context_from_lancedb(dbtable, question, top_k=2)
response = client.chat.completions.create(
    model="phi3:3.8b",
    temperature=0,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational.",
        },
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    ],
)
print(response.choices[0].message.content)

 Based on the information provided in our Employee Handbook, employees receive 30 days of Paid Time Off (PTO) per year. Your PTO accrual starts from your first day with us and it amounts to about 2.5 days each month. After completing your first full year at our company, you'll earn an additional day annually but the total will not exceed 25 days in a year.

However, there are some details that might need further clarification or updates from HR such as how to request PTO and whether it can be transferred between years which is currently disallowed according to our policy. If you have any questions about this information or anything else not covered herein, please feel free to reach out directly to the Human Resources department for assistance!


In [10]:
context

'In this section, we are going to be covering information about paid time off: Employees receive 30 days of\xa0Paid Time Off (PTO)\xa0per year. Your PTO accrual begins the day you join our company and you receive 2.5 days per month. You can take your PTO at any time after your first week with us and you can use time off you haven’t accrued yet. You will earn one additional day per year after your first year with our company, with a cap at 25 days overall. If you want to use PTO, send a request through our HRIS. If your manager or HR approves, you are permitted to take your leave. You do not have to specify a reason for requesting PTO. You cannot transfer any remaining PTO to the next year. We encourage you to use your time off throughout the year. If you leave our company, we may compensate accrued PTO with your final paycheck according to local law. When the law doesn’t have provisions, we will compensate accrued leave to employees who were not terminated for cause.\nThese holidays ar